In [2]:
%load_ext autoreload
%autoreload 2
%run ../draw_functions.py
%run ../kstest.py

In [3]:
import numpy as np
from sqlalchemy import text
from sqlalchemy.engine.url import URL
from sqlalchemy import create_engine
import pandas as pd

In [4]:
conn_url = URL.create(drivername='mssql+pyodbc',
                      host='sqlmelpjol',
                      database='Enterprise',
                      query={
                          "driver":"ODBC Driver 17 for SQL Server",
                          "TrustServerCertificate": "yes",
                          "authentication": "ActiveDirectoryIntegrated"
                      }
                      )

eng = create_engine(conn_url)
conn = eng.connect()

In [5]:
with open(r'..\The_script.sql', 'r') as file:
    sql_query = file.read()

with eng.begin() as con:
    query = text(sql_query)
    df_data2 = pd.read_sql_query(query, conn)

In [8]:
# Create a new column in df_data with default value of NaN
df_data2['fitted_slope'] = np.nan

for sample in df_data2['Sample_Code'].unique():
    df_sample = df_data2[df_data2['Sample_Code'] == sample]
    
    z1 = np.polyfit(df_sample['xtime'], df_sample['slope'], 3)
    p1 = np.poly1d(z1)
    
    # Update the 'fitted_slope' in df_data for the current sample
    df_data2.loc[df_data2['Sample_Code'] == sample, 'fitted_slope'] = p1(df_sample['xtime'])

In [9]:
# I want a new dataframe with three columns: OrderNo, std_dev_batch, std_dev_sample
df_std_dev = pd.DataFrame(columns=['OrderNo', 'std_dev_sample'])
for OrderNo in df_data2['OrderNo'].unique():
    # Trouver le xtime correspondant au max de la pente pour chaque groupe
    df_order = df_data2[df_data2['OrderNo'] == OrderNo]
    
    max_slope_sample_xtime = df_order.loc[df_order.groupby('Sample_Code')['fitted_slope'].idxmax()][['Batch', 'xtime']].set_index('Batch')['xtime']
    
    std_dev_sample = round(np.std(max_slope_sample_xtime)*1000,2)
    
    df_std_dev.loc[len(df_std_dev)] = [OrderNo, std_dev_sample]

In [10]:
df_std_dev

,OrderNo,std_dev_sample
0,MX2240527189601,3.09
1,MX224031409701,15.70
2,MX124010718HY01,16.04
3,MX224021813ZD01,1.90
4,MX224032917GQ01,8.47
...,...,...
820,MX22401301QZ01,3.72
821,MX324021914YE01,10.98
822,MX324060521C401,9.75
823,MX324011214EV01,4.39
